#Import

In [2]:
# 기본 패키지
import re
import time
import pickle
import pandas as pd
from tqdm import tqdm
from tqdm import trange

#경고 무시
import warnings
warnings.filterwarnings('ignore')

# 크롤링 패키지
import requests

#Chorme Driver 설치

In [3]:
!pip install selenium
!apt-get update         #apt install을 정확히 실행하기 위해 ubuntu 업데이트
!apt install chromium-chromedriver

     |████████████████████████████████| 904 kB 9.5 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ub

#Chrome 연결

In [4]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin 

import sys 
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') 

from selenium import webdriver 

chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument('--headless') 
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-dev-shm-usage') 

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options) 

#요기요 홈페이지로 이동
driver.get("https://www.yogiyo.co.kr/mobile/#/%EA%B2%BD%EA%B8%B0%EB%8F%84/445320/")

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


#Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#페이지 한번 맨아래로 내리는 함수

In [5]:
def scroll_bottom():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#음식점의 모든 리뷰 객체를 리턴해주는 함수

In [6]:
def get_all_review_elements():
    reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    return reviews

#리뷰 페이지를 열어주는 함수

In [7]:
def go_to_review():
    print('리뷰 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(2)
    print('리뷰 페이지 로드 완료!')

#리뷰 더보기 클릭해주는 함수

In [8]:
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()
    time.sleep(2)

#리뷰 페이지를 전부 펼쳐주는 함수

In [9]:
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in trange(click_count):
        try:
            scroll_bottom()
            click_more_review()
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 리뷰 불러오기 완료!')

#리뷰 객체를 데이터 프레임으로 변환해주는 함수

In [10]:
def dataframe_make():
    df = pd.DataFrame(columns=['가게이름', '리뷰', '총점', '맛', '양', '배달', '날짜'])

    for review in tqdm(get_all_review_elements()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
        try:
            df.loc[len(df)] = {
                '가게이름':driver.find_element_by_class_name('restaurant-name').text,
                '리뷰':review.find_element_by_css_selector('p').text,
                '총점':str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                '맛':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                '양':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                '배달':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text,
                '날짜':review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text, }
        except Exception as e:
            print('리뷰 페이지 에러')
            print(e)
            pass
            
    return df

In [11]:
for i in range(300005,300007):
    num = str(i).zfill(6)
    
    driver.get('https://www.yogiyo.co.kr/mobile/#/' + num)

    time.sleep(3)

    try:
        go_to_review()
        stretch_review_page()
        df = dataframe_make()
        df.to_csv('/content/drive/MyDrive/yogiyo/data/' + num + '.csv')

    except Exception as e:
        print(e)

리뷰 페이지 로드중...


  0%|          | 0/104 [00:00<?, ?it/s]

리뷰 페이지 로드 완료!
모든 리뷰 불러오기 시작...


  0%|          | 0/1046 [00:00<?, ?it/s]

모든 리뷰 불러오기 완료!


...

  0%|          | 0/14 [00:00<?, ?it/s]

리뷰 페이지 로드 완료!
모든 리뷰 불러오기 시작...


100%|██████████| 14/14 [00:29<00:00,  2.11s/it]


모든 리뷰 불러오기 완료!


100%|██████████| 149/149 [00:26<00:00,  5.53it/s]
